在机器学习中，离散化（Discretization） 是将连续数值特征转换为离散类别特征的过程，而二值化（Binarization） 则是将数值特征转换为 0/1 二值的特殊情况。这两种技术常用于简化模型、增强鲁棒性或处理特定类型的算法（如规则树模型）。下面详细介绍 KBinsDiscretizer 和 Binarizer 的用法、参数及案例。

## 一、KBinsDiscretizer：分箱离散化
1. 核心作用
    将连续数值特征划分为多个离散区间（分箱），每个区间作为一个类别。例如，将年龄分为 “青年”“中年”“老年”。
2. 关键参数

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(
    n_bins=5,                # 每个特征的分箱数量
    encode='onehot',         # 编码方式：'onehot'、'onehot-dense'、'ordinal'
    strategy='quantile',     # 分箱策略：'uniform'、'quantile'、'kmeans'
    dtype=np.float64,        # 输出数据类型
    subsample=1e5,           # 拟合时的最大样本数（内存优化）
    random_state=None        # 随机种子（仅用于kmeans策略）
)

3. 参数详解
* n_bins：
    每个特征的分箱数量，可以是整数（所有特征相同）或数组（为每个特征指定不同数量）。
* encode：
    - 'onehot'（默认）：返回稀疏的独热编码矩阵；
    - 'onehot-dense'：返回密集的独热编码矩阵；
    - 'ordinal'：返回有序整数编码（如 0,1,2...）。
* strategy：
    - 'uniform'：等宽分箱，每个区间宽度相等；
    - 'quantile'：等频分箱，每个区间包含近似相同数量的样本；
    - 'kmeans'：基于 K-means 聚类的分箱，区间边界由聚类中心决定。

In [1]:
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

# 创建示例数据（年龄和收入）
X = np.array([
    [20, 5000],
    [30, 6000],
    [40, 7000],
    [50, 8000],
    [60, 9000]
])

# 创建离散化器（等频分箱，独热编码）
discretizer = KBinsDiscretizer(
    n_bins=[2, 3],          # 年龄分2箱，收入分3箱
    encode='onehot-dense',  # 返回密集矩阵
    strategy='quantile'     # 等频分箱
)

In [2]:
# 拟合并转换数据
X_discretized = discretizer.fit_transform(X)

print("原始数据：")
print(X)
print("\n离散化后的数据：")
print(X_discretized)  # 输出：[[1. 0. 1. 0. 0.], [1. 0. 0. 1. 0.], ...]

原始数据：
[[  20 5000]
 [  30 6000]
 [  40 7000]
 [  50 8000]
 [  60 9000]]

离散化后的数据：
[[1. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [3]:
# 查看分箱边界
print("\n分箱边界：")
for feature_idx, bins in enumerate(discretizer.bin_edges_):
    print(f"特征 {feature_idx} 的分箱边界：{bins}")
# 输出：特征0(年龄)边界：[20. 45. 60.]，特征1(收入)边界：[5000. 6500. 8500. 9000.]


分箱边界：
特征 0 的分箱边界：[20. 40. 60.]
特征 1 的分箱边界：[5000.         6333.33333333 7666.66666667 9000.        ]


In [4]:
# 使用有序编码
discretizer_ordinal = KBinsDiscretizer(
    n_bins=2,
    encode='ordinal',
    strategy='uniform'  # 等宽分箱
)
X_ordinal = discretizer_ordinal.fit_transform(X)
print("\n有序编码结果：")
print(X_ordinal)  # 输出：[[0. 0.], [0. 0.], [1. 1.], [1. 1.], [1. 1.]]


有序编码结果：
[[0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]


## 二、Binarizer：二值化
### 1. 核心作用
    将数值特征转换为 0/1 二值，阈值以上为 1，否则为 0。常用于将连续特征转换为布尔特征（如“是否大于某个值”）
### 2. 关键参数

In [2]:
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(
    threshold=0.0,         # 二值化阈值
    copy=True              # 是否复制数据（避免原地修改）
)

### 3. 参数详解
    阈值，大于该值的样本将被设为 1，否则设为 0。默认值为 0.0。
### 4. 案例代码

In [3]:
import numpy as np
from sklearn.preprocessing import Binarizer

# 创建示例数据（考试分数）
X = np.array([
    [60],
    [70],
    [80],
    [90],
    [50]
])

# 创建二值化器（阈值为75分）
binarizer = Binarizer(threshold=75)

# 转换数据
X_binarized = binarizer.transform(X)

print("原始数据：")
print(X)
print("\n二值化后的数据（阈值=75）：")
print(X_binarized)  # 输出：[[0.], [0.], [1.], [1.], [0.]]

# 对多维数据使用不同阈值
X_multi = np.array([
    [1.0, 2.0],
    [3.0, 4.0],
    [5.0, 6.0]
])

# 分别对每列设置阈值（需先reshape）
binarizer_col1 = Binarizer(threshold=2.0)
binarizer_col2 = Binarizer(threshold=5.0)

X_multi_binarized = np.hstack([
    binarizer_col1.transform(X_multi[:, [0]]),
    binarizer_col2.transform(X_multi[:, [1]])
])

print("\n多维数据二值化：")
print(X_multi_binarized)  # 输出：[[0. 0.], [1. 0.], [1. 1.]]

原始数据：
[[60]
 [70]
 [80]
 [90]
 [50]]

二值化后的数据（阈值=75）：
[[0]
 [0]
 [1]
 [1]
 [0]]

多维数据二值化：
[[0. 0.]
 [1. 0.]
 [1. 1.]]


## 三、两种方法的对比

|特性	|KBinsDiscretizer	|Binarizer|
| ---- | ---- | ---- |
|输出类别数	|每个特征可设置多个分箱（≥2）	|固定为 2 个类别（0 和 1）|
|核心参数	|分箱数量 n_bins、策略 strategy	|阈值 threshold|
|适用场景	|连续特征的多类别离散化（如年龄分组）	|布尔特征生成（如是否超过阈值）|
|对模型的影响	|引入非线性，简化模型复杂度	|增强特征的解释性（非黑即白）|

## 四、实际应用建议

1. KBinsDiscretizer 的策略选择：
    * 等宽分箱（uniform）：适用于分布均匀的数据，但对异常值敏感；
    * 等频分箱（quantile）：更鲁棒，避免了异常值导致的分箱不平衡；
    * K-means 分箱：适合数据分布不规则，需根据数据内在结构分箱的场景。
2. Binarizer 的阈值确定：
    * 可通过业务知识设定（如 “是否超过及格线”）；
    * 或使用数据统计量（如均值、中位数）作为阈值。
3. 与 Pipeline 结合：

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer

preprocessor = ColumnTransformer(
    transformers=[
        ('discretize', KBinsDiscretizer(n_bins=3), ['age', 'income']),
        ('binarize', Binarizer(threshold=0.5), ['score'])
    ]
)

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeClassifier())
])

NameError: name 'DecisionTreeClassifier' is not defined

## 五、总结

* KBinsDiscretizer：通过分箱将连续特征转换为多类别离散特征，可选择不同的分箱策略以适应不同的数据分布。
* Binarizer：通过设定阈值将特征转换为二值特征，常用于简化特征或生成布尔条件。

这两种方法都能在一定程度上增强模型的鲁棒性和解释性，具体选择需根据数据特性和业务需求决定。